In [10]:
import numpy as np
import pandas as pd 
from data.lib import dormant_data

In [11]:
X = dormant_data(path='../../../ZALK/Data_sourcing/compressed/BTC-USDT.parquet',start_date ='2021-01-01 00:00:00',
                 end_date='2021-11-01 00:00:00',datz=True)

ori_df = X.copy()
X = X.drop(columns=['quote_asset_volume','number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume'])

X['date'] = pd.to_datetime(X['date'])
X = X.set_index('date')
X.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
X.head()

              
    ---> Does df have NaN values ? == False
    ---> Total Rows In The Data  ? == 436764
            


,Open,High,Low,Close,Volume
date,,,,,
2021-01-01 00:01:00,28961.669922,29017.500000,28961.009766,29009.910156,58.477501
2021-01-01 00:02:00,29009.539062,29016.710938,28973.580078,28989.300781,42.470329
2021-01-01 00:03:00,28989.679688,28999.849609,28972.330078,28982.689453,30.360678
2021-01-01 00:04:00,28982.669922,28995.929688,28971.800781,28975.650391,24.124338
2021-01-01 00:05:00,28975.650391,28979.529297,28933.160156,28937.109375,22.396013


In [12]:
def triple_barrier(price, ub, lb, vertical_barrier):
    """
    Implements the Triple Barrier Method.
    
    Parameters:
    price (pd.Series): A pandas series of prices.
    ub (float): The upper barrier.
    lb (float): The lower barrier.
    vertical_barrier (pd.Series): A pandas series of vertical barriers.
    
    Returns:
    pd.DataFrame: A pandas dataframe containing the exit events.
    """
    
    # Initialize the exit events dataframe
    events = pd.DataFrame(index=price.index, columns=['stop_loss', 'take_profit'])
    
    # Loop over each row of the price series
    for i in range(1, len(price)):
        
        # Compute the holding period for the position
        holding_period = vertical_barrier[i] - price.index[i]
        
        # Define the upper and lower barriers for the current position
        current_ub = price[i] * (1 + ub)
        current_lb = price[i] * (1 - lb)
        
        # Set the stop loss and take profit levels
        stop_loss   = np.nan
        take_profit = np.nan
        
        # Check if the price hits the upper barrier
        if price[i:i+holding_period].max() >= current_ub:
            take_profit = 1
        
        # Check if the price hits the lower barrier
        elif price[i:i+holding_period].min() <= current_lb:
            stop_loss = -1
        
        # Store the exit events in the events dataframe
        events.loc[price.index[i], 'take_profit'] = take_profit
        events.loc[price.index[i], 'stop_loss'] = stop_loss
        
    return events

In [13]:
price = X['Close'].reset_index(drop=True)

# Define the upper and lower barriers and the vertical barrier
ub = 0.04  ## Take-Profit
lb = 0.04  ## Stop-Loss

min = 720 ## 12 hours
vertical_barrier = pd.Series(price.index + min)

# Apply the Triple Barrier Method
events = triple_barrier(price, ub, lb, vertical_barrier)

In [14]:
events
labels = events['stop_loss'].combine_first(events['take_profit'])
labels = labels.fillna(0)
labels.value_counts()

 0    291012
-1     73784
 1     71968
Name: stop_loss, dtype: int64

In [15]:
X['labels'] = labels.to_numpy()
X.to_csv('data/result.csv',index=False)